We will first install the necessary packages for this project, `folium` for map visualization and `ipywidgets` to better display the dataset.

In [ ]:
!pip install folium
!pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.5 MB/s eta 0:00:00


We then import the necessary libraries: `pandas` for data manipulation and analysis, `folium` for interactive geospatial visualization, `ipywidgets` for creating interactive GUIs in Jupyter notebooks, and `numpy` for numerical computing. Additionally, we use `branca` for advanced mapping functionalities, and various `IPython.display` elements like display and `IFrame` for rich media integration and embedding interactive elements in the notebook.

In [ ]:
import pandas as pd
import folium
import ipywidgets as widgets
import numpy as np

import branca
from IPython.display import display
from branca.element import Element, Template
from folium.plugins import MarkerCluster
from IPython.display import IFrame

Instead of mounting the notebook to the drive and requiring the download of the CSV to the drive, we will be reading the [CSV](https://data.seattle.gov/Community/Age-Friendly-Discount-Directory/ujt3-6w6q/about_data) directly from the internet using the dataset link provided by Seattle Open Data.




In [ ]:
url = 'https://data.seattle.gov/resource/ujt3-6w6q.csv'
data = pd.read_csv(url)
data.head() # Limited in 5 rows

,todays_date,category,org_name,org_name2,street_address,city,state,zip_code,phone_number,discount_email,website,discount_description,discount_description_long,cardholder_requirement,authorization,approved_for_publishing
0,2019-01-24T00:00:00.000,Other,1DayBanner.com,NaN,"1636 E. Edinger Ave, Suite D",Santa Ana,CA,92705,NaN,specialoffers@1daybanner.com,https://www.1daybanner.com,"Enter coupon code ""GOVDEALS"" during checkout f...","Enter coupon code ""GOVDEALS"" during checkout f...",GOLD or FLASH card required,True,True
1,2022-06-13T00:00:00.000,Food & Beverage / Restaurant,A+ Hong Kong Restaurant,NaN,667 S King St.,Seattle,WA,98104,206-602-6008,Aplus-2022@hotmail.com,NaN,5% off,NaN,Both Gold and FLASH card,True,True
2,4/7/2023,Food & Beverage / Restaurant,Aloha Plates,"Endless, LLC",511 S Weller St,Seattle,WA,98104,253-282-0060,aloha@endlessn.com,https://www.alohaplates-sea.com/s/order,10% off entire purchase. Must present Gold or ...,NaN,Both Gold and FLASH card,True,True
3,2022-08-08T00:00:00.000,Retail Shopping,Andaluz,NaN,4908 Rainier Avenue S.,Seattle,WA,98118,206-384-5255,esquivel.karla@gmail.com,https://andaluz.us/,"15% off entire purchase, excluding sale items....",NaN,Both Gold and FLASH card,True,True
4,4/12/2018,Fitness & Recreation; Classes & Learning,"Ballard Pool, Seattle Parks and Recreation",NaN,1471 NW 67th St,Seattle,WA,98117,206-684-4094,NaN,http://www.seattle.gov/parks/find/pools/ballar...,"We offer adult and senior adult lessons, exerc...",Details on senior discounts at: https://www.se...,No card required,True,True


Unfortunately, free geocoding APIs that work with Google Colab, such as Nominatim, are highly prone to connection errors. Instead, we will use the website [geoapify](https://www.geoapify.com/) to geocode our dataset. The CSV provided by the website was then downloaded locally and uploaded to my [GitHub](https://github.com/edwardyeung04/Age_Friendly_Discount_Directory_CSV/blob/main/age_locations.csv) account, where the [raw URL link](https://raw.githubusercontent.com/edwardyeung04/Age_Friendly_Discount_Directory_CSV/main/age_locations.csv) was obtained to be used as the updated dataset.

In [ ]:
new_url = 'https://raw.githubusercontent.com/edwardyeung04/Age_Friendly_Discount_Directory_CSV/main/age_locations.csv'
updated_dataset = pd.read_csv(new_url)

# Deleting locations outside of Washington
updated_dataset = updated_dataset[updated_dataset['state_code'] == 'WA']

# We will edit the dataset so that we delete redundant columns:
columns_to_remove = ['original_Last Updated', 'original_Street Address', 'original_City', 'original_State', 'original_Zip Code',
                     'name', 'housenumber', 'postcode', 'district', 'suburb', 'city', 'county', 'state', 'state_code', 'country_code',
                     'confidence','confidence_city_level','confidence_street_level', 'street', 'country', 'original_Other Organization Name',
                     'original_Discount Description (long)', 'original_Authorization for Release of Information', 'original_Approved for Publishing',
                     'attribution','attribution_license','attribution_url']
updated_dataset.drop(columns=columns_to_remove, inplace=True)

# Changing column names to be more descriptive
updated_dataset.rename(columns={'original_Category': 'Service Category',
                                'original_Organization Name': 'Organization Name',
                                'original_Phone Number': 'Phone Number',
                                'original_E-mail': 'Email',
                                'original_Website': 'Website',
                                'original_Discount Description': 'Discount Description',
                                'original_Cardholder Requirement': 'Cardholder Requirement',
                                'lat': 'Latitude',
                                'lon': 'Longitude',
                                'formatted': 'Address',
                                }, inplace=True)


# Delete rows where there is no latitude and longitude
updated_dataset.dropna(subset=['Latitude', 'Longitude'], inplace=True)

# Instead of using updated_dataset.head(), we alternatively use ipywidgets to display the data in a far more viewable format

# Function to display dataframe based on user inputs
def view_dataframe(head_rows=5):
    display(updated_dataset.head(head_rows))

# Creating a slider widget for row selection
row_slider = widgets.IntSlider(
    value=5,
    min=1,
    max= len(data),  # Display up to 30 rows, or the length of your dataframe if it's shorter
    step=1,
    description='Rows to Show:',
    continuous_update=False
)

# Creating an interactive widget
widgets.interactive(view_dataframe, head_rows=row_slider)

interactive(children=(IntSlider(value=5, continuous_update=False, description='Rows to Show:', max=128, min=1)…

In [ ]:
#Create a Folium map
initial_map = folium.Map(location=[updated_dataset['Latitude'].mean(), updated_dataset['Longitude'].mean()], zoom_start=12)

# Adding markers to the map
for _, row in updated_dataset.iterrows():
     popup_message = '<br>'.join([f'{col}: {row[col]}' for col in updated_dataset.columns if pd.notna(row[col])])
     folium.Marker([row['Latitude'], row['Longitude']], popup=popup_message).add_to(initial_map)

# Display Map
initial_map

As seen above, the map is quite basic, featuring various markers across Washington. Each location has an interactive marker that displays all the information regarding the location. However, this could be vastly improved.

 It would be ideal to color-code each category of services. However, Folium only allows the following colors: 'pink', 'darkgreen', 'gray', 'darkred', 'cadetblue', 'green', 'purple', 'lightblue', 'red', 'beige', 'lightgreen', 'blue', 'lightgray', 'orange', 'darkpurple', 'white', 'lightred', 'darkblue', 'black'. Despite having 19 available colors, there are more categories in the Service Category column than the colors provided by Folium.

In [ ]:
# Count the number of distinct categories in the 'original_Category' column
distinct_entries_count = updated_dataset['Service Category'].nunique()
print("Number of distinct entries:", distinct_entries_count)

Number of distinct entries: 28


Therefore, we will group similar service categories together to streamline the categorization process.

In [ ]:
distinct_entries = updated_dataset['Service Category'].unique()
for entry in distinct_entries:
    print(entry)

# As you can see, some of these categories overlap or have extremely long names

Food & Beverage / Restaurant
Retail Shopping
Fitness & Recreation; Classes & Learning
Health & Medical; Other
Arts, Culture & Entertainment
Hair, Skin & Spa
Utilities & Cable TV
Health & Medical
Home, Garden & Home Repair
Auto Services & Transportation
Legal & Financial
Fitness & Recreation;#Classes & Learning
Classes & Learning; Other
Home, Garden & Home Repair; Retail Shopping
Food & Beverage / Grocery; Food & Beverage / Restaurant
Pets & Veterinary
Hotels, Travel & Tourism; Fitness & Recreation
Fitness & Recreation
Health & Medical; Classes & Learning
Classes & Learning; Home, Garden & Home Repair; Other
Home, Garden & Home Repair; Pets & Veterinary; Retail Shopping
Elder Care; Food & Beverage / Grocery; Health & Medical; Legal & Financial; Other
Technology, Computers & Mobile Phones
Arts, Culture & Entertainment; Classes & Learning; Technology, Computers & Mobile Phones
Classes & Learning
Technology, Computers & Mobile Phones; Other
Cleaning Services
Garden & Home Repair


# To reclassify the categories into fewer groups, we can combine similar categories and simplify the classifications. Here is the reclassification:
---

**1. Food & Beverage:**
- Includes 'Food & Beverage / Restaurant', 'Food & Beverage / Grocery; Food & Beverage / Restaurant'

**2. Retail Shopping:**
- Remains as is.

**3. Fitness & Recreation:**
- Combines 'Fitness & Recreation; Classes & Learning', 'Fitness & Recreation', 'Fitness & Recreation;#Classes & Learning', 'Hotels, Travel & Tourism; Fitness & Recreation'

**4. Health & Medical:**
- Includes 'Health & Medical; Other', 'Health & Medical', 'Health & Medical; Classes & Learning'

**5. Arts, Culture & Entertainment:**
- Combines 'Arts, Culture & Entertainment', 'Arts, Culture & Entertainment; Classes & Learning; Technology, Computers & Mobile Phones'

**6. Personal Care:**
- Includes 'Hair, Skin & Spa'

**7. Utilities & Services:**
- Combines 'Utilities & Cable TV', 'Cleaning Services'

**8. Home & Garden:**
- Combines 'Home, Garden & Home Repair', 'Home, Garden & Home Repair; Retail Shopping', 'Garden & Home Repair', 'Classes & Learning; Home, Garden & Home Repair; Other'

**9. Auto Services & Transportation:**
- Remains as is.

**10. Legal & Financial:**
- Remains as is.

**11. Education & Learning:**
- Combines 'Classes & Learning; Other', 'Classes & Learning', 'Technology, Computers & Mobile Phones; Other', 'Technology, Computers & Mobile Phones'

**12. Pets & Veterinary:**
- Remains as is.

**13. Elder Care & Wellness:**
- Combines 'Elder Care', 'Elder Care; Food & Beverage / Grocery; Health & Medical; Legal & Financial; Other'

**14. Multi-category Services:**
- For complex categories that involve multiple services, like 'Home, Garden & Home Repair; Pets & Veterinary; Retail Shopping'

---

This reclassification reduces the number of categories to 14, making the data more manageable and easier to visualize or analyze.

In [ ]:
# Applying the reclassification

mappings ={'Food & Beverage / Restaurant': 'Food & Beverage',
        'Food & Beverage / Grocery; Food & Beverage / Restaurant': 'Food & Beverage',
        'Retail Shopping': 'Retail Shopping',
        'Fitness & Recreation; Classes & Learning': 'Fitness & Recreation',
        'Fitness & Recreation': 'Fitness & Recreation',
        'Fitness & Recreation;#Classes & Learning': 'Fitness & Recreation',
        'Hotels, Travel & Tourism; Fitness & Recreation': 'Fitness & Recreation',
        'Health & Medical; Other': 'Health & Medical',
        'Health & Medical': 'Health & Medical',
        'Health & Medical; Classes & Learning': 'Health & Medical',
        'Arts, Culture & Entertainment': 'Arts, Culture & Entertainment',
        'Arts, Culture & Entertainment; Classes & Learning; Technology, Computers & Mobile Phones': 'Arts, Culture & Entertainment',
        'Hair, Skin & Spa': 'Personal Care',
        'Utilities & Cable TV': 'Utilities & Services',
        'Cleaning Services': 'Utilities & Services',
        'Home, Garden & Home Repair': 'Home & Garden',
        'Home, Garden & Home Repair; Retail Shopping': 'Home & Garden',
        'Garden & Home Repair': 'Home & Garden',
        'Classes & Learning; Home, Garden & Home Repair; Other': 'Home & Garden',
        'Auto Services & Transportation': 'Auto Services & Transportation',
        'Legal & Financial': 'Legal & Financial',
        'Classes & Learning; Other': 'Education & Learning',
        'Classes & Learning': 'Education & Learning',
        'Technology, Computers & Mobile Phones; Other': 'Education & Learning',
        'Technology, Computers & Mobile Phones': 'Education & Learning',
        'Pets & Veterinary': 'Pets & Veterinary',
        'Elder Care': 'Elder Care & Wellness',
        'Elder Care; Food & Beverage / Grocery; Health & Medical; Legal & Financial; Other': 'Elder Care & Wellness',
        'Home, Garden & Home Repair; Pets & Veterinary; Retail Shopping': 'Multi-category Services',
        }

updated_dataset['Service Category'] = updated_dataset['Service Category'].replace(mappings)

distinct_entries = updated_dataset['Service Category'].unique()
for entry in distinct_entries:
    print(entry)


Food & Beverage
Retail Shopping
Fitness & Recreation
Health & Medical
Arts, Culture & Entertainment
Personal Care
Utilities & Services
Home & Garden
Auto Services & Transportation
Legal & Financial
Education & Learning
Pets & Veterinary
Multi-category Services
Elder Care & Wellness


This code then creates an interactive map using Folium in Python, where 14 service categories are color-coded for easy identification. Each category, ranging from 'Food & Beverage' to 'Multi-category Services', is assigned a unique color. The map is centered on the average latitude and longitude of the dataset, and a MarkerCluster is used to handle multiple locations. For each location, a marker is added with a popup displaying detailed information, color-coded based on its service category and modified using HTML code for better viewing and additional functionality. Additionally, a custom legend is added to the map, indicating the color associated with each service category.

In [ ]:
# Define a color for each of the 14 categories
category_colors = {
    'Food & Beverage': 'darkred',
    'Retail Shopping': 'blue',
    'Fitness & Recreation': 'darkgreen',
    'Health & Medical': 'pink',
    'Arts, Culture & Entertainment': 'purple',
    'Personal Care': 'orange',
    'Utilities & Services': 'cadetblue',
    'Home & Garden': 'green',
    'Auto Services & Transportation': 'darkblue',
    'Legal & Financial': 'black',
    'Education & Learning': 'beige',
    'Pets & Veterinary': 'lightblue',
    'Elder Care & Wellness': 'lightgreen',
    'Multi-category Services': 'gray'
}

# Create a Folium map with a standard tile
map = folium.Map(location=[updated_dataset['Latitude'].mean(), updated_dataset['Longitude'].mean()],
                 tiles='OpenStreetMap',
                 zoom_start=12)

# Initialize MarkerCluster
marker_cluster = MarkerCluster().add_to(map)

# Adding color-coded markers to the map inside the MarkerCluster
for _, row in updated_dataset.iterrows():
    if pd.notna(row['Latitude']) and pd.notna(row['Longitude']):
        # Using HTML and CSS to style the popup message
        popup_message = """
        <div style="font-family: Arial, sans-serif; font-size: 14px; padding: 10px; border-radius: 5px; background: #f9f9f9; box-shadow: 1px 1px 2px rgba(0,0,0,0.2);">
        <h4 style="margin-top: 0;">Location Details</h4>
        """

        for col in updated_dataset.columns:
            # Skip Latitude and Longitude in the popup
            if col in ['Latitude', 'Longitude']:
                continue

            if pd.notna(row[col]):
                value = row[col]
                # Check if the column is 'Website' and format as a clickable link
                if col == 'Website' and value.startswith('http'):
                    value = f'<a href="{value}" target="_blank" style="color: #007bff;">{value}</a>'
                popup_message += f'<p style="margin: 5px 0; line-height: 1.5;"><b>{col}:</b> {value}</p>'

        # Add a Directions link
        if pd.notna(row['Address']):
            maps_url = f"https://www.google.com/maps/dir/?api=1&destination={row['Address'].replace(' ', '+')}"
            popup_message += f'<p style="margin: 5px 0; line-height: 1.5;"><b>Directions:</b> <a href="{maps_url}" target="_blank" style="color: #007bff;">Click Here</a></p>'

        popup_message += '</div>'

        marker_color = category_colors.get(row['Service Category'], 'gray')  # Default to gray if category not found
        folium.Marker(
            [row['Latitude'], row['Longitude']],
            popup=folium.Popup(popup_message, max_width=450),
            icon=folium.Icon(color=marker_color)
        ).add_to(marker_cluster)

# Create custom icons for the legend labels
legend_labels = {
    'Food & Beverage': 'Food & Beverage',
    'Retail Shopping': 'Retail Shopping',
    'Fitness & Recreation': 'Fitness & Recreation',
    'Health & Medical': 'Health & Medical',
    'Arts, Culture & Entertainment': 'Arts, Culture & Entertainment',
    'Personal Care': 'Personal Care',
    'Utilities & Services': 'Utilities & Services',
    'Home & Garden': 'Home & Garden',
    'Auto Services & Transportation': 'Auto Services & Transportation',
    'Legal & Financial': 'Legal & Financial',
    'Education & Learning': 'Education & Learning',
    'Pets & Veterinary': 'Pets & Veterinary',
    'Elder Care & Wellness': 'Elder Care & Wellness',
    'Multi-category Services': 'Multi-category Services'
}

# Add a custom JavaScript legend to the map
legend_html = """
<div style="position: fixed;
     bottom: 50px; left: 50px; width: 200px; height: 250px; overflow-y: auto;
     background-color: white; border: 2px solid grey; z-index:9999; font-size:12px; padding: 5px;">
"""

for label, color in category_colors.items():
    legend_html += f'<i class="fa fa-map-marker" style="color:{color}"></i> {legend_labels[label]}<br>'

legend_html += "</div>"

map.get_root().html.add_child(folium.Element(legend_html))

# Display the Map
map

# Thanks for viewing!